In [1]:
import numpy as np
import pandas as pd
import re
import newspaper
import nltk
import random
from transformers import pipeline
import torch
nltk.download('punkt')

C:\Users\Sethuraman T V\.conda\envs\IITM_Ford\lib\site-packages\torchaudio\extension\extension.py:13: UserWarning: torchaudio C++ extension is not available.
  warnings.warn('torchaudio C++ extension is not available.')
C:\Users\Sethuraman T V\.conda\envs\IITM_Ford\lib\site-packages\torchaudio\backend\utils.py:89: UserWarning: No audio backend is available.
  warnings.warn('No audio backend is available.')
[nltk_data] Downloading package punkt to C:\Users\Sethuraman T
[nltk_data]     V\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# df=pd.read_csv(r"C:\Users\Sethuraman T V\Documents\misinformation_mit.csv")
# df=pd.read_csv(r"C:\Users\Sethuraman T V\Downloads\geosim\misinformation\covidb_50.csv")
df=pd.read_csv(r"C:\Users\Sethuraman T V\Downloads\geosim\misinformation\CoviDB50_Sample_Send.csv")
df.shape

(20, 5)

In [3]:
def set_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [4]:
def get_url_text_csv(input_csv=None):
    url_text_list=[]
    df=pd.read_csv(input_csv)
    for url in (df['URL']):
        url=str(url)
        url_i = newspaper.Article(url="%s" % (url), language='en')
        try:
            url_i.download()
            url_i.parse()
            # print(url_i.text)
            url_text_list.append(url_i.text)
            print("URL: {} Text downloaded.".format(url))
        except:
            url_text_list.append("Not Accessible")
            print("URL: {} is not accessible".format(url))
    print("Function Executed")
    return url_text_list

In [5]:
# url_text_list=get_url_text_csv(r"C:\Users\Sethuraman T V\Downloads\geosim\misinformation\covidb_50.csv")
url_text_list=get_url_text_csv(r"C:\Users\Sethuraman T V\Downloads\geosim\misinformation\CoviDB50_Sample_Send.csv")
# df=pd.read_csv(r"C:\Users\Sethuraman T V\Downloads\geosim\misinformation\covidb.csv")
df["URL_Text"]=url_text_list

URL: https://www.cdc.gov/coronavirus/2019-ncov/communication/responder-stories/MacarenaGarcia.html Text downloaded.
URL: https://www.cdc.gov/coronavirus/2019-ncov/vaccines/recommendations/specific-groups/teachers-childcare.html Text downloaded.
URL: https://phgkb.cdc.gov/PHGKB/coVInfoClip.action?action=home Text downloaded.
URL: https://www.cdc.gov/coronavirus/2019-ncov/lab/naats.html Text downloaded.
URL: https://www.cdc.gov/coronavirus/2019-ncov/vaccines/recommendations/underlying-conditions.html Text downloaded.
URL: https://www.cdc.gov/coronavirus/2019-ncov/testing/self-testing.html Text downloaded.
URL: https://www.thelancet.com/journals/lancet/article/PIIS0140-6736(21)01064-3/fulltext Text downloaded.
URL: https://www.astrazeneca.com/content/astraz/media-centre/articles/2021/astrazenecas-covid-19-vaccine-european-union-supply-commitment.html Text downloaded.
URL: https://investors.modernatx.com/sec-filings/sec-filing/4/0001127602-21-016223 Text downloaded.
URL: https://www.nature

In [6]:
def clean_text(text):
    text = text.encode("ascii", errors="ignore").decode("ascii") #remove non-ascii, Chinese characters
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\n\n", " ", text)
    text = re.sub(r"\W", " ", text)
    text = re.sub(r"^\d+\s|\s\d+\s|\s\d+$", " ", text)
    text = text.strip(" ")
    text = re.sub(r"[^\w\s]", "", text)
    text = re.sub(' +',' ', text).strip() # get rid of multiple spaces and replace with a single
    return text

df["Clean_Text"] = df['URL_Text'].map(lambda text: clean_text(text))

In [7]:
df.to_csv("Summaries_Result_1.csv")

In [8]:
class Summerization():
    def __init__(self,csv_name=None):
        self.csv_name=csv_name
        self.url=None
        self.url_i=None
        self.url_newspaper_3k_summary_list=[]

    def newspaper3k_summary(self):
        self.df=pd.read_csv(self.csv_name)
        for self.url in self.df['URL']:
            self.url=str(self.url)
            try:
                self.url_i = newspaper.Article(url="%s" % (self.url), language='en')
                self.url_i.download()
                self.url_i.html
                self.url_i.parse()
                self.url_i.nlp()
                self.url_newspaper_3k_summary_list.append(self.url_i.summary)
                print("NewsPaper 3k summary executed for url {}".format(self.url))
            except:
                self.url_newspaper_3k_summary_list.append("URL Not available")
                print("NewsPaper 3k summary Not executed for url {}".format(self.url))
        print("Get article summary from ,Function executed")
        return self.url_newspaper_3k_summary_list
    

In [9]:
df.to_csv("Summary_output_1.csv")

In [11]:
# get_summary=Summerization(r"C:\Users\Sethuraman T V\Documents\misinformation_mit.csv")
get_summary=Summerization(r"C:\Users\Sethuraman T V\Downloads\geosim\misinformation\CoviDB50_Sample_Send.csv")
url_newspaper_3k_summary_list=get_summary.newspaper3k_summary()
df["URL_Summary_Newspaper3k"]=url_newspaper_3k_summary_list

NewsPaper 3k summary executed for url https://www.cdc.gov/coronavirus/2019-ncov/communication/responder-stories/MacarenaGarcia.html
NewsPaper 3k summary executed for url https://www.cdc.gov/coronavirus/2019-ncov/vaccines/recommendations/specific-groups/teachers-childcare.html
NewsPaper 3k summary executed for url https://phgkb.cdc.gov/PHGKB/coVInfoClip.action?action=home
NewsPaper 3k summary executed for url https://www.cdc.gov/coronavirus/2019-ncov/lab/naats.html
NewsPaper 3k summary executed for url https://www.cdc.gov/coronavirus/2019-ncov/vaccines/recommendations/underlying-conditions.html
NewsPaper 3k summary executed for url https://www.cdc.gov/coronavirus/2019-ncov/testing/self-testing.html
NewsPaper 3k summary executed for url https://www.thelancet.com/journals/lancet/article/PIIS0140-6736(21)01064-3/fulltext
NewsPaper 3k summary executed for url https://www.astrazeneca.com/content/astraz/media-centre/articles/2021/astrazenecas-covid-19-vaccine-european-union-supply-commitment.

In [12]:
# Adding a word-count column for easier reference
df['Word_Count'] = df['Clean_Text'].str.count(' ') + 1

In [13]:
crit1 = df["Word_Count"] < 700
short_speeches = df[crit1].copy()
short_speeches.shape

(10, 7)

In [15]:
corpus_short = list(short_speeches['Clean_Text'].values)
nlp_summarizer = pipeline(
    "summarization")

In [16]:
short_speeches["Summary"] = nlp_summarizer(corpus_short)

In [17]:
short_speeches['HF_1_Summary_Text'] = [x.get('summary_text') for x in short_speeches['Summary']]

In [18]:
short_speeches=short_speeches.drop(['Summary'],axis=1)

In [19]:
result = pd.merge(df, short_speeches, how="outer")

In [20]:
result

,URL,URL_Summary_Newspaper3k,Word_Count,HF_Bart_Summary,HF_T5_Summary,URL_Text,Clean_Text,HF_1_Summary_Text
0,https://www.cdc.gov/coronavirus/2019-ncov/comm...,Now she boasts a doctorate degree in public he...,811,macarena garcia has a doctorate degree in pub...,macarena garcia has a doctorate degree in pub...,CDC’s Macarena Garcia compares COVID-19 case t...,cdcs macarena garcia compares covid case trend...,NaN
1,https://www.cdc.gov/coronavirus/2019-ncov/vacc...,"What Teachers, School Staff, and Childcare Wor...",319,everyone years of age and older is now eligib...,covid vaccines are safe and effective a covid ...,CDC now recommends that people aged 65 years a...,cdc now recommends that people aged years and ...,Everyone years of age and older is now eligib...
2,https://phgkb.cdc.gov/PHGKB/coVInfoClip.action...,Pathogen and Human Genomics StudiesImpact of s...,1436,vaccine efficacy against covid was 3 confiden...,vaccine efficacy against covid was 3 confiden...,Pathogen and Human Genomics Studies\n\nImpact ...,pathogen and human genomics studies impact of ...,NaN
3,https://www.cdc.gov/coronavirus/2019-ncov/lab/...,NAATs for SARS-CoV-2 test specimens from eithe...,635,A nucleic acid amplification test or naat is ...,NaN,Summary of Recent Changes\n\nUpdates as of Jun...,summary of recent changes updates as of june c...,A nucleic acid amplification test or naat is ...
4,https://www.cdc.gov/coronavirus/2019-ncov/vacc...,COVID-19 vaccines may be administered to most ...,236,people who have previously had autoimmune con...,no data are currently available on the safety ...,CDC now recommends that people aged 65 years a...,cdc now recommends that people aged years and ...,cdc now recommends that people aged years and...
5,https://www.cdc.gov/coronavirus/2019-ncov/test...,OverviewNote: If your COVID-19 test is positiv...,731,If you have symptoms or have had a known expo...,NaN,Overview\n\nNote: If your COVID-19 test is pos...,overview note if your covid test is positive t...,NaN
6,https://www.thelancet.com/journals/lancet/arti...,Treatment of COVID-19 patients with convalesce...,1142,As the covid pandemic continues researchers c...,As the covid pandemic continues researchers c...,1 Joyner MJ\n\nCarter RE\n\nSenefeld JW\n\net ...,joyner mj carter re senefeld jw et al convales...,NaN
7,https://www.astrazeneca.com/content/astraz/med...,We understand and share in the frustration tha...,379,Astrazeneca is committed to delivering billio...,astrazeneca is committed to delivering billion...,AstraZeneca is committed to delivering billion...,astrazeneca is committed to delivering billion...,Astrazeneca is committed to delivering billio...
8,https://investors.modernatx.com/sec-filings/se...,"You are now leaving investors.modernatx.com, a...",67,external websites are not controlled by moder...,moderna assumes no responsibility for the info...,"You are now leaving investors.modernatx.com, a...",you are now leaving investors modernatx com a ...,external websites are not controlled by moder...
9,https://www.nature.com/articles/s41375-021-012...,Search strategy and selection criteriaWe condu...,405,A systematic review and meta analysis focused...,systematic review and meta analysis focused on...,Search strategy and selection criteria\n\nWe c...,search strategy and selection criteria we cond...,A systematic review and meta analysis focused...


In [21]:
crit2=df["Word_Count"]>=700
long_speech=df[2:3].copy()
long_speech_list = df[crit2].copy()
long_speech_list['Clean_Text']
long_speech_list.shape

(10, 7)

In [22]:
def split_text(text, limit, sep=" "):
    words = text.split()
    if max(map(len, words)) > limit:
        raise ValueError("limit is too small")
    res, part, others = [], words[0], words[1:]
    for word in others:
        if len(sep)+len(word) > limit-len(part):
            res.append(part)
            part = word
        else:
            part += sep+word
    if part:
        res.append(part)
    return res

In [23]:
list_1=[]
for i in range(len(long_speech_list)):
    # print(long_speech_list[i:i+1])
    # print(long_speech_list[i:i+1]['Clean_Text'].map(lambda x: split_text(x, limit=2500)).copy())
    # print(long_speech_list[i:i+1]['Clean_Text'])
    text_temp=long_speech_list[i:i+1]['Clean_Text']
    # print(text_temp.map(lambda x: split_text(x, limit=2500)).copy())
    temp=text_temp.map(lambda x: split_text(x, limit=2500)).copy()
    # print(temp[1:])
    # print(temp)
    print("temp:",temp)
    list_1.append(temp)

long_speech_list['Split_Text']=list_1

temp: 0    [cdcs macarena garcia compares covid case tren...
Name: Clean_Text, dtype: object
temp: 2    [pathogen and human genomics studies impact of...
Name: Clean_Text, dtype: object
temp: 5    [overview note if your covid test is positive ...
Name: Clean_Text, dtype: object
temp: 6    [joyner mj carter re senefeld jw et al convale...
Name: Clean_Text, dtype: object
temp: 11    [health workers administer sars cov tests at a...
Name: Clean_Text, dtype: object
temp: 13    [given the relative novelty of covid prognosti...
Name: Clean_Text, dtype: object
temp: 15    [demographic and clinical characteristics of a...
Name: Clean_Text, dtype: object
temp: 16    [to our knowledge this is one of the largest n...
Name: Clean_Text, dtype: object
temp: 18    [more than 000 people in brazil have died of c...
Name: Clean_Text, dtype: object
temp: 19    [inactivation of spop ck1 signaling destabiliz...
Name: Clean_Text, dtype: object


In [24]:
appended_data = []
for row in long_speech_list['Split_Text'].index:
    long_speech_split = (
    pd.DataFrame(long_speech_list['Split_Text'][row].explode())
    .stack()
    .reset_index()
    .rename(columns={0: "Sections"})
    .drop(["level_0", "level_1"], axis=1)
    )
    corpus_long = list(long_speech_split['Sections'].values)
    long_speech_split["Summary"] = nlp_summarizer(corpus_long)
    long_speech_split['HF_Summary_Text'] = [x.get('summary_text') for x in long_speech_split['Summary']]
    long_speech_split['HF_Summary_Text']
    long_speech_split['long_speech_id']=row
    long_speech_split=long_speech_split.groupby('long_speech_id')['HF_Summary_Text'].apply(' '.join).reset_index()
    print(long_speech_split)
    appended_data.append(long_speech_split)
appended_data = pd.concat(appended_data)

   long_speech_id                                    HF_Summary_Text
0               0   macarena garcia has a doctorate degree in pub...
   long_speech_id                                    HF_Summary_Text
0               2   9 3693 patients admitted for sars cov infecti...
   long_speech_id                                    HF_Summary_Text
0               5   If you have symptoms or have had a known expo...
   long_speech_id                                    HF_Summary_Text
0               6   As the covid pandemic continues researchers c...
   long_speech_id                                    HF_Summary_Text
0              11   The number of new covid cases in sri lanka wh...
   long_speech_id                                    HF_Summary_Text
0              13   There was evidence of hepatocellular injury a...
   long_speech_id                                    HF_Summary_Text
0              15   demographic and clinical characteristics of a...
   long_speech_id                 

In [25]:
appended_data 

,long_speech_id,HF_Summary_Text
0,0,macarena garcia has a doctorate degree in pub...
0,2,9 3693 patients admitted for sars cov infecti...
0,5,If you have symptoms or have had a known expo...
0,6,As the covid pandemic continues researchers c...
0,11,The number of new covid cases in sri lanka wh...
0,13,There was evidence of hepatocellular injury a...
0,15,demographic and clinical characteristics of a...
0,16,A nationwide retrospective cohort study ident...
0,18,Report released on may was assembled by a pan...
0,19,inactivation of spop ck1 signaling destabiliz...


In [26]:
result_2 = pd.merge(result, appended_data,right_on='long_speech_id', left_index=True,how="outer")

In [27]:
result_2

,URL,URL_Summary_Newspaper3k,Word_Count,HF_Bart_Summary,HF_T5_Summary,URL_Text,Clean_Text,HF_1_Summary_Text,long_speech_id,HF_Summary_Text
0.0,https://www.cdc.gov/coronavirus/2019-ncov/comm...,Now she boasts a doctorate degree in public he...,811,macarena garcia has a doctorate degree in pub...,macarena garcia has a doctorate degree in pub...,CDC’s Macarena Garcia compares COVID-19 case t...,cdcs macarena garcia compares covid case trend...,NaN,0,macarena garcia has a doctorate degree in pub...
NaN,https://www.cdc.gov/coronavirus/2019-ncov/vacc...,"What Teachers, School Staff, and Childcare Wor...",319,everyone years of age and older is now eligib...,covid vaccines are safe and effective a covid ...,CDC now recommends that people aged 65 years a...,cdc now recommends that people aged years and ...,Everyone years of age and older is now eligib...,1,NaN
0.0,https://phgkb.cdc.gov/PHGKB/coVInfoClip.action...,Pathogen and Human Genomics StudiesImpact of s...,1436,vaccine efficacy against covid was 3 confiden...,vaccine efficacy against covid was 3 confiden...,Pathogen and Human Genomics Studies\n\nImpact ...,pathogen and human genomics studies impact of ...,NaN,2,9 3693 patients admitted for sars cov infecti...
NaN,https://www.cdc.gov/coronavirus/2019-ncov/lab/...,NAATs for SARS-CoV-2 test specimens from eithe...,635,A nucleic acid amplification test or naat is ...,NaN,Summary of Recent Changes\n\nUpdates as of Jun...,summary of recent changes updates as of june c...,A nucleic acid amplification test or naat is ...,3,NaN
NaN,https://www.cdc.gov/coronavirus/2019-ncov/vacc...,COVID-19 vaccines may be administered to most ...,236,people who have previously had autoimmune con...,no data are currently available on the safety ...,CDC now recommends that people aged 65 years a...,cdc now recommends that people aged years and ...,cdc now recommends that people aged years and...,4,NaN
0.0,https://www.cdc.gov/coronavirus/2019-ncov/test...,OverviewNote: If your COVID-19 test is positiv...,731,If you have symptoms or have had a known expo...,NaN,Overview\n\nNote: If your COVID-19 test is pos...,overview note if your covid test is positive t...,NaN,5,If you have symptoms or have had a known expo...
0.0,https://www.thelancet.com/journals/lancet/arti...,Treatment of COVID-19 patients with convalesce...,1142,As the covid pandemic continues researchers c...,As the covid pandemic continues researchers c...,1 Joyner MJ\n\nCarter RE\n\nSenefeld JW\n\net ...,joyner mj carter re senefeld jw et al convales...,NaN,6,As the covid pandemic continues researchers c...
NaN,https://www.astrazeneca.com/content/astraz/med...,We understand and share in the frustration tha...,379,Astrazeneca is committed to delivering billio...,astrazeneca is committed to delivering billion...,AstraZeneca is committed to delivering billion...,astrazeneca is committed to delivering billion...,Astrazeneca is committed to delivering billio...,7,NaN
NaN,https://investors.modernatx.com/sec-filings/se...,"You are now leaving investors.modernatx.com, a...",67,external websites are not controlled by moder...,moderna assumes no responsibility for the info...,"You are now leaving investors.modernatx.com, a...",you are now leaving investors modernatx com a ...,external websites are not controlled by moder...,8,NaN
NaN,https://www.nature.com/articles/s41375-021-012...,Search strategy and selection criteriaWe condu...,405,A systematic review and meta analysis focused...,systematic review and meta analysis focused on...,Search strategy and selection criteria\n\nWe c...,search strategy and selection criteria we cond...,A systematic review and meta analysis focused...,9,NaN


Model_3

In [28]:
crit3 = df["Word_Count"] < 470

shorter_speeches = df[crit3].copy()
shorter_speeches.shape

(7, 7)

In [29]:
corpus_shorter = list(shorter_speeches['Clean_Text'].values)

In [30]:
t5_summarizer = pipeline(
    "summarization",
    model="t5-base",
    tokenizer="t5-base"
)

In [31]:
shorter_speeches["T5_Summary"] = t5_summarizer(corpus_shorter)

Token indices sequence length is longer than the specified maximum sequence length for this model (543 > 512). Running this sequence through the model will result in indexing errors


In [32]:
shorter_speeches['T5_Summary_Text'] = [x.get('summary_text') for x in shorter_speeches['T5_Summary']]
shorter_speeches

,URL,URL_Summary_Newspaper3k,Word_Count,HF_Bart_Summary,HF_T5_Summary,URL_Text,Clean_Text,T5_Summary,T5_Summary_Text
1,https://www.cdc.gov/coronavirus/2019-ncov/vacc...,"What Teachers, School Staff, and Childcare Wor...",319,everyone years of age and older is now eligib...,covid vaccines are safe and effective a covid ...,CDC now recommends that people aged 65 years a...,cdc now recommends that people aged years and ...,{'summary_text': 'cdc recommends that people a...,cdc recommends that people aged years and olde...
4,https://www.cdc.gov/coronavirus/2019-ncov/vacc...,COVID-19 vaccines may be administered to most ...,236,people who have previously had autoimmune con...,no data are currently available on the safety ...,CDC now recommends that people aged 65 years a...,cdc now recommends that people aged years and ...,{'summary_text': 'cdc recommends that people a...,cdc recommends that people aged years and olde...
7,https://www.astrazeneca.com/content/astraz/med...,We understand and share in the frustration tha...,379,Astrazeneca is committed to delivering billio...,astrazeneca is committed to delivering billion...,AstraZeneca is committed to delivering billion...,astrazeneca is committed to delivering billion...,{'summary_text': 'astrazeneca is committed to ...,astrazeneca is committed to delivering billion...
8,https://investors.modernatx.com/sec-filings/se...,"You are now leaving investors.modernatx.com, a...",67,external websites are not controlled by moder...,moderna assumes no responsibility for the info...,"You are now leaving investors.modernatx.com, a...",you are now leaving investors modernatx com a ...,{'summary_text': 'moderna assumes no responsib...,moderna assumes no responsibility for the info...
9,https://www.nature.com/articles/s41375-021-012...,Search strategy and selection criteriaWe condu...,405,A systematic review and meta analysis focused...,systematic review and meta analysis focused on...,Search strategy and selection criteria\n\nWe c...,search strategy and selection criteria we cond...,{'summary_text': 'systematic review and meta a...,systematic review and meta analysis focused on...
14,https://www.nature.com/articles/s41375-021-012...,"Furthermore, patients with CLL experience subo...",330,Only half of vaccinated patients with cll dev...,only half of vaccinated patients with cll deve...,Given that immunocompromised patients were exc...,given that immunocompromised patients were exc...,{'summary_text': 'only half of vaccinated pati...,only half of vaccinated patients with cll deve...
17,https://www.nature.com/articles/d41591-021-000...,Dancing Man / Alamy Stock PhotoA proportion of...,190,A proportion of patients with covid will requ...,a proportion of patients with covid will requi...,You can also search for this author in PubMed ...,you can also search for this author in pubmed ...,{'summary_text': 'a proportion of patients wit...,a proportion of patients with covid will requi...


In [33]:
shorter_speeches=shorter_speeches.drop(['T5_Summary'],axis=1)

In [34]:
result_3 = pd.merge(result_2, shorter_speeches, how="outer")

In [35]:
crit4=df["Word_Count"]>=470
long_speech=df[2:3].copy()
long_speech_list_2 = df[crit4].copy()
long_speech_list_2['Clean_Text']
long_speech_list_2.shape

(13, 7)

In [36]:
list_2=[]
for i in range(len(long_speech_list_2)):
    # print(long_speech_list[i:i+1])
    # print(long_speech_list[i:i+1]['Clean_Text'].map(lambda x: split_text(x, limit=2500)).copy())
    # print(long_speech_list[i:i+1]['Clean_Text'])
    text_temp=long_speech_list_2[i:i+1]['Clean_Text']
    # print(text_temp.map(lambda x: split_text(x, limit=2500)).copy())
    temp=text_temp.map(lambda x: split_text(x, limit=2500)).copy()
    # print(temp[1:])
    # print(temp)
    print("temp:",temp)
    list_2.append(temp)

long_speech_list_2['Split_Text']=list_2

temp: 0    [cdcs macarena garcia compares covid case tren...
Name: Clean_Text, dtype: object
temp: 2    [pathogen and human genomics studies impact of...
Name: Clean_Text, dtype: object
temp: 3    [summary of recent changes updates as of june ...
Name: Clean_Text, dtype: object
temp: 5    [overview note if your covid test is positive ...
Name: Clean_Text, dtype: object
temp: 6    [joyner mj carter re senefeld jw et al convale...
Name: Clean_Text, dtype: object
temp: 10    [the potential for healthy gums to reduce the ...
Name: Clean_Text, dtype: object
temp: 11    [health workers administer sars cov tests at a...
Name: Clean_Text, dtype: object
temp: 12    [facing a limited vaccine supply the united ki...
Name: Clean_Text, dtype: object
temp: 13    [given the relative novelty of covid prognosti...
Name: Clean_Text, dtype: object
temp: 15    [demographic and clinical characteristics of a...
Name: Clean_Text, dtype: object
temp: 16    [to our knowledge this is one of the largest n...
Nam

In [37]:
appended_data_2 = []
for row in long_speech_list_2['Split_Text'].index:
    long_speech_split_2 = (
    pd.DataFrame(long_speech_list_2['Split_Text'][row].explode())
    .stack()
    .reset_index()
    .rename(columns={0: "Sections"})
    .drop(["level_0", "level_1"], axis=1)
    )
    corpus_long = list(long_speech_split_2['Sections'].values)
    long_speech_split_2["T5_Summary"] = nlp_summarizer(corpus_long)
    long_speech_split_2['HF_T5_Summary_Text'] = [x.get('summary_text') for x in long_speech_split_2['T5_Summary']]
    long_speech_split_2['HF_T5_Summary_Text']
    long_speech_split_2['long_speech_id_2']=row
    long_speech_split_2=long_speech_split_2.groupby('long_speech_id_2')['HF_T5_Summary_Text'].apply(' '.join).reset_index()
    print(long_speech_split_2)
    appended_data_2.append(long_speech_split_2)

appended_data_2 = pd.concat(appended_data_2)

   long_speech_id_2                                 HF_T5_Summary_Text
0                 0   macarena garcia has a doctorate degree in pub...
   long_speech_id_2                                 HF_T5_Summary_Text
0                 2   9 3693 patients admitted for sars cov infecti...
   long_speech_id_2                                 HF_T5_Summary_Text
0                 3   A nucleic acid amplification test or naat is ...
   long_speech_id_2                                 HF_T5_Summary_Text
0                 5   If you have symptoms or have had a known expo...
   long_speech_id_2                                 HF_T5_Summary_Text
0                 6   As the covid pandemic continues researchers c...
   long_speech_id_2                                 HF_T5_Summary_Text
0                10   The potential for healthy gums to reduce the ...
   long_speech_id_2                                 HF_T5_Summary_Text
0                11   The number of new covid cases in sri lanka wh...
   lon

In [38]:
appended_data_2

,long_speech_id_2,HF_T5_Summary_Text
0,0,macarena garcia has a doctorate degree in pub...
0,2,9 3693 patients admitted for sars cov infecti...
0,3,A nucleic acid amplification test or naat is ...
0,5,If you have symptoms or have had a known expo...
0,6,As the covid pandemic continues researchers c...
0,10,The potential for healthy gums to reduce the ...
0,11,The number of new covid cases in sri lanka wh...
0,12,A study suggests that delaying the second dos...
0,13,There was evidence of hepatocellular injury a...
0,15,demographic and clinical characteristics of a...


In [39]:
result_3 = pd.merge(result_3, appended_data_2,right_on='long_speech_id_2', left_index=True,how="outer")

In [40]:
result_3

,URL,URL_Summary_Newspaper3k,Word_Count,HF_Bart_Summary,HF_T5_Summary,URL_Text,Clean_Text,HF_1_Summary_Text,long_speech_id,HF_Summary_Text,T5_Summary_Text,long_speech_id_2,HF_T5_Summary_Text
0.0,https://www.cdc.gov/coronavirus/2019-ncov/comm...,Now she boasts a doctorate degree in public he...,811,macarena garcia has a doctorate degree in pub...,macarena garcia has a doctorate degree in pub...,CDC’s Macarena Garcia compares COVID-19 case t...,cdcs macarena garcia compares covid case trend...,NaN,0,macarena garcia has a doctorate degree in pub...,NaN,0,macarena garcia has a doctorate degree in pub...
NaN,https://www.cdc.gov/coronavirus/2019-ncov/vacc...,"What Teachers, School Staff, and Childcare Wor...",319,everyone years of age and older is now eligib...,covid vaccines are safe and effective a covid ...,CDC now recommends that people aged 65 years a...,cdc now recommends that people aged years and ...,Everyone years of age and older is now eligib...,1,NaN,cdc recommends that people aged years and olde...,1,NaN
0.0,https://phgkb.cdc.gov/PHGKB/coVInfoClip.action...,Pathogen and Human Genomics StudiesImpact of s...,1436,vaccine efficacy against covid was 3 confiden...,vaccine efficacy against covid was 3 confiden...,Pathogen and Human Genomics Studies\n\nImpact ...,pathogen and human genomics studies impact of ...,NaN,2,9 3693 patients admitted for sars cov infecti...,NaN,2,9 3693 patients admitted for sars cov infecti...
0.0,https://www.cdc.gov/coronavirus/2019-ncov/lab/...,NAATs for SARS-CoV-2 test specimens from eithe...,635,A nucleic acid amplification test or naat is ...,NaN,Summary of Recent Changes\n\nUpdates as of Jun...,summary of recent changes updates as of june c...,A nucleic acid amplification test or naat is ...,3,NaN,NaN,3,A nucleic acid amplification test or naat is ...
NaN,https://www.cdc.gov/coronavirus/2019-ncov/vacc...,COVID-19 vaccines may be administered to most ...,236,people who have previously had autoimmune con...,no data are currently available on the safety ...,CDC now recommends that people aged 65 years a...,cdc now recommends that people aged years and ...,cdc now recommends that people aged years and...,4,NaN,cdc recommends that people aged years and olde...,4,NaN
0.0,https://www.cdc.gov/coronavirus/2019-ncov/test...,OverviewNote: If your COVID-19 test is positiv...,731,If you have symptoms or have had a known expo...,NaN,Overview\n\nNote: If your COVID-19 test is pos...,overview note if your covid test is positive t...,NaN,5,If you have symptoms or have had a known expo...,NaN,5,If you have symptoms or have had a known expo...
0.0,https://www.thelancet.com/journals/lancet/arti...,Treatment of COVID-19 patients with convalesce...,1142,As the covid pandemic continues researchers c...,As the covid pandemic continues researchers c...,1 Joyner MJ\n\nCarter RE\n\nSenefeld JW\n\net ...,joyner mj carter re senefeld jw et al convales...,NaN,6,As the covid pandemic continues researchers c...,NaN,6,As the covid pandemic continues researchers c...
NaN,https://www.astrazeneca.com/content/astraz/med...,We understand and share in the frustration tha...,379,Astrazeneca is committed to delivering billio...,astrazeneca is committed to delivering billion...,AstraZeneca is committed to delivering billion...,astrazeneca is committed to delivering billion...,Astrazeneca is committed to delivering billio...,7,NaN,astrazeneca is committed to delivering billion...,7,NaN
NaN,https://investors.modernatx.com/sec-filings/se...,"You are now leaving investors.modernatx.com, a...",67,external websites are not controlled by moder...,moderna assumes no responsibility for the info...,"You are now leaving investors.modernatx.com, a...",you are now leaving investors modernatx com a ...,external websites are not controlled by moder...,8,NaN,moderna assumes no responsibility for the info...,8,NaN
NaN,https://www.nature.com/articles/s41375-021-012...,Search strategy and selection criteriaWe condu...,405,A systematic review and meta ana

In [41]:
""""
make names int
merge summaries for short and long 
url and 3 summaries 
summary_2= 1 args - return the chosen sum 
for now hardcode one value 
the out return as a list 
"""

'"\nmake names int\nmerge summaries for short and long \nurl and 3 summaries \nsummary_2= 1 args - return the chosen sum \nfor now hardcode one value \nthe out return as a list \n'

In [44]:
result_4=result_3.copy()

In [45]:
result_4=result_4.drop(['long_speech_id','long_speech_id_2'],axis=1)

In [46]:
# result_4
result_4['HF_Bart_Summary']=result_4['HF_1_Summary_Text'].combine_first(result_4['HF_Summary_Text'])
result_4['HF_T5_Summary']=result_4['T5_Summary_Text'].combine_first(result_4['HF_T5_Summary_Text'])
result_4=result_4.drop(['HF_1_Summary_Text','HF_Summary_Text','T5_Summary_Text','HF_T5_Summary_Text'],axis=1)


In [47]:
result_4

,URL,URL_Summary_Newspaper3k,Word_Count,HF_Bart_Summary,HF_T5_Summary,URL_Text,Clean_Text
0.0,https://www.cdc.gov/coronavirus/2019-ncov/comm...,Now she boasts a doctorate degree in public he...,811,macarena garcia has a doctorate degree in pub...,macarena garcia has a doctorate degree in pub...,CDC’s Macarena Garcia compares COVID-19 case t...,cdcs macarena garcia compares covid case trend...
NaN,https://www.cdc.gov/coronavirus/2019-ncov/vacc...,"What Teachers, School Staff, and Childcare Wor...",319,Everyone years of age and older is now eligib...,cdc recommends that people aged years and olde...,CDC now recommends that people aged 65 years a...,cdc now recommends that people aged years and ...
0.0,https://phgkb.cdc.gov/PHGKB/coVInfoClip.action...,Pathogen and Human Genomics StudiesImpact of s...,1436,9 3693 patients admitted for sars cov infecti...,9 3693 patients admitted for sars cov infecti...,Pathogen and Human Genomics Studies\n\nImpact ...,pathogen and human genomics studies impact of ...
0.0,https://www.cdc.gov/coronavirus/2019-ncov/lab/...,NAATs for SARS-CoV-2 test specimens from eithe...,635,A nucleic acid amplification test or naat is ...,A nucleic acid amplification test or naat is ...,Summary of Recent Changes\n\nUpdates as of Jun...,summary of recent changes updates as of june c...
NaN,https://www.cdc.gov/coronavirus/2019-ncov/vacc...,COVID-19 vaccines may be administered to most ...,236,cdc now recommends that people aged years and...,cdc recommends that people aged years and olde...,CDC now recommends that people aged 65 years a...,cdc now recommends that people aged years and ...
0.0,https://www.cdc.gov/coronavirus/2019-ncov/test...,OverviewNote: If your COVID-19 test is positiv...,731,If you have symptoms or have had a known expo...,If you have symptoms or have had a known expo...,Overview\n\nNote: If your COVID-19 test is pos...,overview note if your covid test is positive t...
0.0,https://www.thelancet.com/journals/lancet/arti...,Treatment of COVID-19 patients with convalesce...,1142,As the covid pandemic continues researchers c...,As the covid pandemic continues researchers c...,1 Joyner MJ\n\nCarter RE\n\nSenefeld JW\n\net ...,joyner mj carter re senefeld jw et al convales...
NaN,https://www.astrazeneca.com/content/astraz/med...,We understand and share in the frustration tha...,379,Astrazeneca is committed to delivering billio...,astrazeneca is committed to delivering billion...,AstraZeneca is committed to delivering billion...,astrazeneca is committed to delivering billion...
NaN,https://investors.modernatx.com/sec-filings/se...,"You are now leaving investors.modernatx.com, a...",67,external websites are not controlled by moder...,moderna assumes no responsibility for the info...,"You are now leaving investors.modernatx.com, a...",you are now leaving investors modernatx com a ...
NaN,https://www.nature.com/articles/s41375-021-012...,Search strategy and selection criteriaWe condu...,405,A systematic review and meta analysis focused...,systematic review and meta analysis focused on...,Search strategy and selection criteria\n\nWe c...,search strategy and selection criteria we cond...


In [45]:
result_4.to_csv("CoviDBrandom50_Sample_Send.csv")